<a href="https://colab.research.google.com/github/prayan2702/Momentum-Screen/blob/main/Copy_of_RajMomnFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Created on Sat Dec 31 2022

@author: Chintan Shah
"""

import numpy as np
import pandas as pd

def getMedianVolume(data):
	return(round(data.median(),0))

def getDailyReturns(data):
	return(data.pct_change())

def getAbsReturns(data):
	x = (data.iloc[-1]/data.iloc[0] - 1)*100
	return(round(x, 2))

def getVolatility(data):
	return(round(np.std(data) * np.sqrt(252) * 100, 2))

def getMonthlyPrices(data):
	grps = data.groupby([data.index.year, data.index.month])
	monthlyPrices = pd.DataFrame()
	for k in grps:
		monthlyPrices = pd.concat([monthlyPrices, k[1].tail(1)])
		# monthlyPrices = monthlyPrices.append(k[1].tail(1))
	return monthlyPrices

def getMonthlyReturns(data):
	return(data.pct_change())

def getSharpe(data):
	return(round(np.sqrt(252) * data.mean()/data.std(), 2))

def getSortino(data):
	return(np.sqrt(252) * data.mean()/data[data<0].std())

def getMaxDrawdown(data):
	cummRet = (data+1).cumprod()
	peak = cummRet.expanding(min_periods = 1).max()
	drawdown = (cummRet/peak) - 1
	return drawdown.min()

def getCalmar(data):
	return(data.mean()*252/abs(getMaxDrawdown(data)))

def getAbsMomentumVolAdjusted(absReturn, volatility):
	return(absReturn/volatility)

def getNMonthRoC(data, N):
	ret = round((data.iloc[-1]/data.iloc[-1-N] - 1) * 100, 2)
	return(ret)

def getNWeekRoC(data, N):
	ret = round((data.iloc[-1]/data.iloc[-1-N] - 1) * 100, 2)
	return(ret)

def getFIP(data):
	retPos = np.sum(data.pct_change()[1:] > 0)
	retNeg = np.sum(data.pct_change()[1:] < 0)
	return(retPos - retNeg)

def getSharpeRoC(roc, volatility):
	return(round(roc/volatility, 2))

def getBeta(dfNifty, data12M):

	dailyReturns = getDailyReturns(pd.concat([dfNifty, data12M], axis = 1))[1:]

	var = dailyReturns.loc[:, 'Nifty'].var()

	cov = dailyReturns.cov()

	cols = cov.columns[1:]

	beta = []

	for k in cols:
		beta.append(round(cov.loc[k, 'Nifty']/var, 2))

	return beta


In [2]:
import pandas as pd
import numpy as np
import time
from scipy.stats import zscore
from datetime import datetime
import yfinance as yf
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt



pd.set_option('display.max_rows', 500)

universe = ['Nifty50', 'Nifty100', 'Nifty200', 'Nifty250', 'Nifty500', 'All', 'AllNSE']

# Pick universe, lookback
U = universe[6] #Selecting All NSE Stocks

# Two-factor ranking
# First sort by Average sharpe of 12M, 9M, 6M and 3M and get the rank
# Next sort by deducting 1M ROC from 12M ROC to avoid recency bias and get the rank
# Add the above 2 ranks and sort by ascending ranks
#rm = ['sharpe12MRoC', 'FIP12M']
#rm = ['averageSharpe', '12MROCMinus1MROC']
rm = ['averageSharpe']

# Apply various filters
applyFilter = True

print('Universe: ', U)
print('Lookback: 12 Months')
print('Ranking:', rm)
print('Filter:', str(applyFilter))
print("\n")

# Dates: shift by 1 month at every rebalance
dates = dict()
# change the start date below to a year from today's date
dates['startDate'] = datetime.strptime('2023-03-26', '%Y-%m-%d')
# change the end date below to today
dates['endDate'] = datetime.strptime('2024-03-28', '%Y-%m-%d')
# calculating dates 1M back, 3M back, 6M back and 9M back
dates['date1M'] = dates['endDate'] - relativedelta(months=+1)
dates['date3M'] = dates['endDate'] - relativedelta(months=+3)
dates['date6M'] = dates['endDate'] - relativedelta(months=+6)
dates['date9M'] = dates['endDate'] - relativedelta(months=+9)

#Print the dates
print('Start Date: ', dates['startDate'])
print('9M Date: ', dates['date9M'])
print('6M Date: ', dates['date6M'])
print('3M Date: ', dates['date3M'])
print('1M Date:', dates['date1M'])
print('End Date: ', dates['endDate'])
print('\n')

# Read index file
if U == 'Nifty50':
    df = pd.read_csv('/content/ind_nifty50list.csv')
elif U == 'Nifty100':
    df = pd.read_csv('/content/ind_nifty100list.csv')
elif U == 'Nifty200':
    df = pd.read_csv('/content/ind_nifty200list.csv')
elif U == 'Nifty250':
    df = pd.read_csv('/content/ind_niftysmallcap250list.csv')
elif U == 'Nifty500':
    df = pd.read_csv('/content/ind_nifty500list.csv')
elif U == 'All':
    df = pd.read_csv('/content/ind_niftytotalmarket_list.csv')
elif U == 'AllNSE':
    df = pd.read_csv('/content/NSE_EQ_ALL.csv')

# Drop unnecessary columns
df.drop(['Series', 'ISIN Code', 'Industry'], axis = 1, inplace = True)

# Create Yahoo symbol
df['Yahoo_Symbol'] = df.Symbol + '.NS'
df = df.set_index('Yahoo_Symbol')
symbol = list(df.index)

# Get yahoofinance data in chunks of 50
CHUNK = 50
close = []
volume = []
for k in range(0, len(symbol), CHUNK):
    print(k, end = ',', flush = True)
    print("\n")
    _symlist = symbol[k:k+CHUNK]
    _x = yf.download(_symlist, start = dates['startDate'], progress = False)
    close = _x['Close'] if len(close) == 0 else pd.concat([close, _x['Close']], axis = 1)
    volume = _x['Close']*_x['Volume'] if len(volume) == 0 else pd.concat([volume, (_x['Close']*_x['Volume'])], axis = 1)
    time.sleep(.5)

print("Done with downloading DATA \n")
# Keep the data within the date range for 1 year data
data12M = close[:dates['endDate']].copy()
volume12M = volume[:dates['endDate']].copy()


# At least 12 months of trading is required
#data12M.dropna(axis = 1, inplace = True)
#volume12M.dropna(axis = 1, inplace = True)
data9M = data12M[dates['date9M']:].copy() # Gets last 9M data
data6M = data12M[dates['date6M']:].copy() # Gets last 6M data
data3M = data12M[dates['date3M']:].copy() # Gets last 3M data
data1M = data12M[dates['date1M']:].copy() # Gets last 1M data

print("Got data slices for each timeframe \n")

# Stats DataFrame
dfStats = pd.DataFrame()

# Rate of change
dfStats['roc1M'] = getAbsReturns(data1M)
dfStats['roc3M'] = getAbsReturns(data3M)
dfStats['roc6M'] = getAbsReturns(data6M)
dfStats['roc9M'] = getAbsReturns(data9M)
dfStats['roc12M'] = getAbsReturns(data12M)

print("Done calculating ROCs \n")


# Volatility
dfStats['volatility3M'] = getVolatility(getDailyReturns(data3M))
dfStats['volatility6M'] = getVolatility(getDailyReturns(data6M))
dfStats['volatility9M'] = getVolatility(getDailyReturns(data9M))
dfStats['volatility12M'] = getVolatility(getDailyReturns(data12M))

print("Done calculating Volatility \n")

dfStats['sharpe12MRoC'] = getSharpeRoC(dfStats['roc12M'], dfStats['volatility12M'])
dfStats['sharpe9MRoC'] = getSharpeRoC(dfStats['roc9M'], dfStats['volatility9M'])
dfStats['sharpe6MRoC'] = getSharpeRoC(dfStats['roc6M'], dfStats['volatility6M'])
dfStats['sharpe3MRoC'] = getSharpeRoC(dfStats['roc3M'], dfStats['volatility3M'])

#MAIN FACTORS for ranking stocks. we are using 2 factors
dfStats['averageSharpe'] = dfStats[["sharpe12MRoC","sharpe9MRoC", "sharpe6MRoC","sharpe3MRoC"]].mean(axis=1) #1st Factor
#dfStats['12MROCMinus1MROC'] = dfStats["roc12M"] - dfStats["roc1M"] #2nd Factor

print("Done calculating Sharpe  \n")

dfStats['volume'] = getMedianVolume(volume12M)

# close and ema
dfStats['Close'] = round(data12M.iloc[-1], 2)
#fileName = './OutputMonthly/RAW_' + dates['endDate'].strftime('%Y-%m-%d') + '_' + U + '_' + '12M' + '_lookback' + '.csv'
#data12M.to_csv(fileName, index = False, float_format = "%.2f")
data12M.fillna(0, inplace=True)
dfStats['dma100d'] = round(data12M.rolling(window=100).mean().iloc[-1], 2)
dfStats['dma200d'] = round(data12M.rolling(window=200).mean().iloc[-1], 2)

#PRINT OUT ALL THE STOCKS DATA without any filters
print("\n Writing unfiltered data to file. Shape in next line \n")
print(dfStats.shape)
fileName = '/content/Unfiltered' + dates['endDate'].strftime('%Y-%m-%d') + '_' + U + '_' + '12M' + '_lookback' + '.csv'
unfiltered = dfStats.reset_index().rename(columns = {'index':'symbol'})
unfiltered.to_csv(fileName, index = False, float_format = "%.2f")

# Get Nifty data (Additional added condition)
dfNifty = yf.download("^NSEI", start = dates['startDate'], progress = False)['Close']
dfNifty = dfNifty.to_frame().rename(columns = {'Close':'Nifty'})

# Get stock beta (Additional added condition)
dfStats['Beta'] = getBeta(dfNifty, data12M)

#Ignore(drop) Top 10 percent Highest Beta stock against Nifty from the selected universe (Additional added condition)
dfStats = dfStats.sort_values(by = ["Beta"], axis = 0, ascending=False)
dfStats = dfStats.tail(len(dfStats) - round(len(dfStats)*0.1))


# Apply filters: e.g. RoC12M > FD rate
cond1 = dfStats['volume'] > 10000000 #volume filter
cond2 = dfStats['Close'] > dfStats['dma100d'] # above 100-day DMA
cond3 = dfStats['Close'] > dfStats['dma200d'] # above 200-day DMA
cond4 = dfStats['roc12M'] > 7 #12M ROC above G-sec rate
cond = cond1 & cond2 & cond3 & cond4

if applyFilter == True:
    filtered = dfStats[cond]
elif applyFilter == False:
    filtered = dfStats


print("Applied Filters \n")

#FIRST, sort by average Sharpe for 12M, 9M, 6M and 3M
output = filtered.sort_values(rm[0], ascending = False).reset_index().rename(columns = {'index':'symbol'})
# Assign rank
output['Rank1'] = range(1, output.shape[0] + 1)

#SECOND, sort by (12M ROC - 1M ROC)
#output = output.sort_values(rm[1], ascending = False)
# Assign rank
#output['Rank2'] = range(1, output.shape[0] + 1)

#Now, combine both the ranks and sort with ranks sorted in ascending order
#output['FinalRank'] = output['Rank1'] + output['Rank2']
output['FinalRank'] = output['Rank1']
output = output.sort_values('FinalRank', ascending = True)

print("\n Writing data to file \n")

fileName = '/content/Filtered' + dates['endDate'].strftime('%Y-%m-%d') + '_' + U + '_' + '12M' + '_lookback' + '.csv'
output.to_csv(fileName, index = False, float_format = "%.2f")

print("\n Successfully Done \n")

Universe:  AllNSE
Lookback: 12 Months
Ranking: ['averageSharpe']
Filter: True


Start Date:  2023-03-26 00:00:00
9M Date:  2023-06-28 00:00:00
6M Date:  2023-09-28 00:00:00
3M Date:  2023-12-28 00:00:00
1M Date: 2024-02-28 00:00:00
End Date:  2024-03-28 00:00:00


0,

50,

100,

150,

200,

250,

300,

350,

400,

450,

500,

550,

600,

650,

700,

750,

800,

850,

900,

950,

1000,

1050,

1100,

1150,

1200,

1250,

1300,

1350,

1400,

1450,

1500,

1550,

1600,

Done with downloading DATA 

Got data slices for each timeframe 

Done calculating ROCs 

Done calculating Volatility 

Done calculating Sharpe  


 Writing unfiltered data to file. Shape in next line 

(1641, 18)
Applied Filters 


 Writing data to file 


 Successfully Done 

